# Grid box - Backtesting

### Import Library

In [76]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [77]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [78]:
data = dataset.copy()

In [79]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-11-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [80]:
min(data['Close'])

862.3

In [81]:
max(data['Close'])

1575.6

In [82]:
def generate_grid(bot, top, grid_distance):
    return (np.arange(bot, top, grid_distance))

In [83]:
bot_grid = 860
top_grid = 1575

In [84]:
grid_distance = 5
grid = generate_grid(bot_grid, top_grid, grid_distance)
grid

array([ 860,  865,  870,  875,  880,  885,  890,  895,  900,  905,  910,
        915,  920,  925,  930,  935,  940,  945,  950,  955,  960,  965,
        970,  975,  980,  985,  990,  995, 1000, 1005, 1010, 1015, 1020,
       1025, 1030, 1035, 1040, 1045, 1050, 1055, 1060, 1065, 1070, 1075,
       1080, 1085, 1090, 1095, 1100, 1105, 1110, 1115, 1120, 1125, 1130,
       1135, 1140, 1145, 1150, 1155, 1160, 1165, 1170, 1175, 1180, 1185,
       1190, 1195, 1200, 1205, 1210, 1215, 1220, 1225, 1230, 1235, 1240,
       1245, 1250, 1255, 1260, 1265, 1270, 1275, 1280, 1285, 1290, 1295,
       1300, 1305, 1310, 1315, 1320, 1325, 1330, 1335, 1340, 1345, 1350,
       1355, 1360, 1365, 1370, 1375, 1380, 1385, 1390, 1395, 1400, 1405,
       1410, 1415, 1420, 1425, 1430, 1435, 1440, 1445, 1450, 1455, 1460,
       1465, 1470, 1475, 1480, 1485, 1490, 1495, 1500, 1505, 1510, 1515,
       1520, 1525, 1530, 1535, 1540, 1545, 1550, 1555, 1560, 1565, 1570])

In [85]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-06 14:15:00,1339.7,1339.9,1338.1,1338.9,8052
2024-11-06 14:20:00,1338.9,1339.4,1338.1,1339.1,4755
2024-11-06 14:25:00,1339.2,1340.3,1339.0,1339.3,7570


In [86]:
def prepare_data(data):
    data = data[data.index > '2024-01-01 00:00:00']
    data['min_OC'] = data.apply(
        lambda r: min(r['Open'], r['Close']), axis=1)
    data['max_OC'] = data.apply(
        lambda r: max(r['Open'], r['Close']), axis=1)
    return data

def cross_grid_line(min_oc, max_oc, price_grid):
    cross_line = 0
    for line in price_grid:
        if min_oc < line < max_oc:
            cross_line = line
            break
    return cross_line

In [87]:
class GridBacktest(Strategy):
    grid_distance = 10
    def init(self):
        super().init()
        self.grid = generate_grid(bot_grid, top_grid, self.grid_distance)

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 27:
            if self.position.is_long or self.position.is_short:
                self.position.close()
                return

        current_min_oc = self.data.min_OC[-1]
        current_max_oc = self.data.max_OC[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        line_cross = cross_grid_line(current_min_oc, current_max_oc, self.grid)
        if line_cross:
            if not self.position:
                if close_price > open_price:
                    ab_range = close_price - line_cross
                    be_range = line_cross - open_price
                    if ab_range < grid_distance / 3:
                        sl = line_cross - (grid_distance / 3)
                        next_line = line_cross + grid_distance
                        tp = next_line - 0.3
                        self.buy(size=1, sl=sl, tp=tp, limit=close_price)
                elif close_price < open_price:
                    ab_range = open_price - line_cross
                    be_range = line_cross - close_price
                    if be_range < grid_distance / 3:
                        sl = line_cross + (grid_distance / 3)
                        prev_line = line_cross - grid_distance
                        tp = prev_line + 0.3
                        self.sell(size=1, sl=sl, tp=tp, limit=close_price)
            

In [88]:
prepared_data = prepare_data(data)
bt = Backtest(prepared_data, GridBacktest, commission=.003, exclusive_orders=False)

In [89]:
stats = bt.run()
# bt.plot()
stats

Start                     2024-01-02 09:00:00
End                       2024-11-06 14:45:00
Duration                    309 days 05:45:00
Exposure Time [%]                   21.422594
Equity Final [$]                  8708.945367
Equity Peak [$]                       10000.0
Return [%]                         -12.910546
Buy & Hold Return [%]               17.745193
Return (Ann.) [%]                   -15.15394
Volatility (Ann.) [%]                0.985535
Sharpe Ratio                       -15.376352
Sortino Ratio                       -9.811828
Calmar Ratio                        -1.168599
Max. Drawdown [%]                  -12.967616
Avg. Drawdown [%]                  -12.967616
Max. Drawdown Duration      309 days 05:35:00
Avg. Drawdown Duration      309 days 05:35:00
# Trades                                  328
Win Rate [%]                        21.036585
Best Trade [%]                       1.093272
Worst Trade [%]                     -1.462394
Avg. Trade [%]                    

In [90]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,3,18,1144.7239,1138.333333,-6.390567,-0.005583,2024-01-02 09:15:00,2024-01-02 10:30:00,None,0 days 01:15:00
1,1,71,75,1134.2927,1134.700000,0.407300,0.000359,2024-01-03 10:40:00,2024-01-03 11:00:00,None,0 days 00:20:00
2,1,122,126,1153.6506,1154.700000,1.049400,0.000910,2024-01-04 10:40:00,2024-01-04 11:00:00,None,0 days 00:20:00
3,1,131,138,1164.8842,1164.700000,-0.184200,-0.000158,2024-01-04 11:25:00,2024-01-04 13:25:00,None,0 days 02:00:00
4,-1,155,157,1156.0215,1161.666667,-5.645167,-0.004883,2024-01-05 09:10:00,2024-01-05 09:20:00,None,0 days 00:10:00
...,...,...,...,...,...,...,...,...,...,...,...
323,-1,10606,10612,1324.9133,1325.300000,-0.386700,-0.000292,2024-11-04 09:20:00,2024-11-04 09:50:00,None,0 days 00:30:00
324,-1,10640,10642,1315.3421,1321.666667,-6.324567,-0.004808,2024-11-04 13:35:00,2024-11-04 13:45:00,None,0 days 00:10:00
325,-1,10651,10652,1316.0400,1318.500000,-2.460000,-0.001869,2024-11-04 14:30:00,2024-11-04 14:45:00,None,0 days 00:15:00
326,1,10703,10704,1322.9570,1325.000000,2.043000,0.001544,2024-11-05 14:45:00,2024-11-06 09:00:00,None,0 days 18:15:00


In [91]:
optimize_stats, heatmap = bt.optimize(grid_distance=range(5, 15, 1),
                    maximize='Return [%]', max_tries=400,
                    random_state=0,
                    return_heatmap=True)

In [92]:
optimize_stats

Start                     2024-01-02 09:00:00
End                       2024-11-06 14:45:00
Duration                    309 days 05:45:00
Exposure Time [%]                   16.922362
Equity Final [$]                  9040.024067
Equity Peak [$]                       10000.0
Return [%]                          -9.599759
Buy & Hold Return [%]               17.745193
Return (Ann.) [%]                  -11.258182
Volatility (Ann.) [%]                0.877325
Sharpe Ratio                       -12.832392
Sortino Ratio                       -9.081592
Calmar Ratio                        -1.167406
Max. Drawdown [%]                   -9.643759
Avg. Drawdown [%]                   -9.643759
Max. Drawdown Duration      309 days 03:40:00
Avg. Drawdown Duration      309 days 03:40:00
# Trades                                  244
Win Rate [%]                        21.311475
Best Trade [%]                       0.454944
Worst Trade [%]                     -0.697524
Avg. Trade [%]                    

In [93]:
# Convert multiindex series to dataframe
heatmap_df = heatmap.unstack()
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_df, annot=True, cmap='viridis', fmt='.0f')
plt.show()

ValueError: zero-size array to reduction operation maximum which has no identity